In [0]:
%pip install --upgrade pip

Python interpreter will be restarted.
  Using cached pip-22.3.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:
      Successfully uninstalled pip-21.2.4
Python interpreter will be restarted.


In [0]:
!pip install nltk
%matplotlib inline

In [0]:
# installing libs

import nltk
#nltk.download()

import pandas as pd
from pandas.core.arrays.sparse import dtype
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.metrics import accuracy_score
from matplotlib import pyplot

import string
import re 
from sklearn.feature_extraction.text import CountVectorizer  #(in case we decided to use count method)
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split  # for spliting - training and testing data
from sklearn.model_selection import KFold, cross_val_score  #Kfold method for validation 
from sklearn.ensemble import RandomForestClassifier  # random forest 

In [0]:
final_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/serenaparve@gwu.edu/final_df/save_df_fil_final.csv") # reading file

In [0]:
 print(final_df.count())

1660


In [0]:
import pandas as pd

In [0]:
df = final_df.toPandas()

In [0]:
print(df.dtypes)

video_id                      object
trending_date                 object
title                         object
channel_title                 object
category_id                   object
text_lemmatized_ngram_join    object
tags                          object
views                         object
likes                         object
dislikes                      object
comment_count                 object
length                        object
dtype: object


In [0]:
df['likes'] = df['likes'].astype('int64')
df['dislikes'] = df['dislikes'].astype('int64')
df['views'] = df['views'].astype('int64')

In [0]:
import math

In [0]:
df['popularity_count'] = df.views * (df.likes - df.dislikes)
df['popularity_log'] = np.log(df.views * (df.likes - df.dislikes))

/databricks/python/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/databricks/python/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [0]:
df = df[['video_id', 'title', 'popularity_count', 'popularity_log', 'text_lemmatized_ngram_join', 'length', 'trending_date', 'category_id', 'tags', 'views', 'likes', 'dislikes', 'comment_count']]

In [0]:
df.head(2)

,video_id,title,popularity_count,popularity_log,text_lemmatized_ngram_join,length,trending_date,category_id,tags,views,likes,dislikes,comment_count
0,0XuOkYlUWIs,13 reasons why season 2 official trailer h...,666206308922,27.224865,hannah one 13 reason season 2 may 18th 2018 wa...,1273.0,18.10.05,24,netflix trailer netflix original series ...,4145678,166732,6033,16437
1,oSEeK9yDNQI,clash royale meet the rascals new card,1034901176120,27.665327,rascal done nownew card soon subscribe merch s...,2222.0,18.11.05,20,clash royale clash royale game supercell...,6142430,175223,6739,10127


In [0]:
print ("final = ", df.shape)

final =  (1660, 13)


## NLP

In [0]:
from sklearn.feature_extraction.text import CountVectorizer  #(in case we decided to use count method)
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split  # for spliting - training and testing data
from sklearn.model_selection import KFold, cross_val_score  #Kfold method for validation 
from sklearn.ensemble import RandomForestClassifier  # random forest """

### New Pred.
Here we just take input for new prediction and prepare it to use later in out models

In [0]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stopword = nltk.corpus.stopwords.words('english')
import re
from string import punctuation

### function for new _ pred

In [0]:
def new_pred_clean():
    new_pred = input("Enter your description: ")
    
    #REMOVING THE LINKS
    new_pred = new_pred.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')

    #EXTRACTING LINKS FROM THE STRING
    new_pred = re.sub(r"https?://\S+", "", new_pred)

    #Remove Tags
    new_pred = re.sub(r"<a[^>]*>(.*?)</a>", r"\1", new_pred)

    #Remove Punctuations
    new_pred = re.sub(f"[{re.escape(punctuation)}]", "", new_pred)
    new_pred = re.sub(r"[^A-Za-z0-9\s]+", "", new_pred)

    #remove single digits
    new_pred = " ".join([w for w in new_pred.split() if not w.isdigit()])

    #remove short words
    tokens = new_pred.split()
    clean_tokens = [t for t in tokens if len(t) > 1]
    new_pred = " ".join(clean_tokens)

    #tokenzing
    def tokenize(text):
        tokens = re.split('\W+', text)
        return tokens
    new_pred = tokenize(new_pred)

    #remove stopwords
    def remove_stopwrods (tokenized_list):
        text = [word for word in tokenized_list if word not in stopword]
        return text
    new_pred = remove_stopwrods(new_pred)

    ## Lemmatizing 
    wn = nltk.WordNetLemmatizer()

    def lemmatizing (tokenized_text):
        text = " ".join([wn.lemmatize(word) for word in tokenized_text])
        return text
    new_pred = lemmatizing(new_pred)
    new_pred_array = [new_pred]
    
    return new_pred
    return new_pred_array

new_pred = new_pred_clean()

Enter your description:  RM 'seoul (prod. HONNE)' Lyric Video    Get 'mono.' on: Google: http://bit.ly/2PKyi96 Mediafire: http://bit.ly/2J9DVeB Dropbox: http://bit.ly/2R4LyFW    Credits:   Director : YongSeok Choi (Lumpens) Assistant Director : Guzza, HyeJeong Park, MinJe Jeong (Lumpens) Director of Photography : HyunWoo Nam(GDW) Composition : Joonki Min @ Sunshine Underground    BigHit Entertainment. All rights reserved.  Unauthorized reproduction is a violation of applicable laws.  Manufactured by BigHit Entertainment, Seoul, Korea.     Connect with BTS: http://www.ibighit.com  http://twitter.com/BTS_bighit  http://twitter.com/BTS_twt  https://www.facebook.com/ibighit http://www.facebook.com/bangtan.official http://instagram.com/BTS.bighitofficial http://weibo.com/BTSbighit   #RM #seoul #mono #BTS

### Bag of Words

In [0]:
ngram_vect = CountVectorizer (ngram_range=(1,1))
x_count_ngram_lemm = pd.DataFrame (ngram_vect.fit_transform (df['text_lemmatized_ngram_join']).toarray())
print (x_count_ngram_lemm.shape)
print (ngram_vect.get_feature_names())
x_count_ngram_lemm.columns =ngram_vect.get_feature_names()
x_count_ngram_lemm.head ()

(1660, 22464)
['00', '000', '000fps', '000mah', '00am', '00pm', '01', '011', '02', '0211have', '02new', '03', '04', '0410', '05', '05062018series', '06', '07', '07062018series', '08', '08pm', '09', '09717072065', '09811946306', '09821297834', '099', '09traveling', '10', '100', '1000', '10005503', '10008740', '1000fps', '10019released', '10037341', '10037342', '10037343', '10037344', '10037345', '10037346', '10041959', '10053660', '10053666', '10053673', '10053683', '10074086', '10080709', '10080725', '10080727', '10080742', '10093711', '100k', '101', '10102320', '10110895', '10140258', '10141660', '10146922', '10146924', '10151856', '10151874', '10151881', '102', '10212204', '10212205', '10229357', '10229358', '10229359', '10229360', '10241097', '10241098', '10241099', '10241117', '10268440', '10268441', '10268444', '10268445', '10268547', '10293642', '10293679', '10293684', '102notoutteaser', '103', '10312882', '10312884', '10312885', '10332196', '10332198', '10332199', '10332200', '1

,00,000,000fps,000mah,00am,00pm,01,011,02,0211have,...,zsuzsanna,zubaan,zuckerberg,zuniga,zur,zurita,zwadlo,zx110,zyaada,zyll
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [0]:
# lib for spliting data
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [0]:
# spliting data 80/20% whole
x_train, x_test, y_train, y_test = train_test_split(x_count_ngram_lemm, df['category_id'], test_size=0.2)

###Random Forest on BoW

In [0]:
# fitting the model whole
rf_1=RandomForestClassifier(n_estimators=50, max_depth=20, n_jobs=-1)
rf_model = rf_1.fit(x_train, y_train)

In [0]:
# get the most important feature - apperantly 1) body_text_len and 2) num_words are amonge the most important features -- WHOLE
sorted (zip(rf_model.feature_importances_,x_train.columns), reverse=True) [0:20]
#  1)text_punct% 2) text_len 3) num_sentences 4) num words are all contributing to the accuracy of the model and considered as important features

Out[81]: [(0.016604222712441965, 'music'),
 (0.012770776828506412, 'album'),
 (0.009096724533535, 'record'),
 (0.008388597480886517, 'recording'),
 (0.008047542636304425, 'available'),
 (0.00783976649414904, 'stream'),
 (0.0075480386701458615, 'llc'),
 (0.006774737414488181, 'video'),
 (0.006700123860179052, 'watch'),
 (0.006657092761855189, 'itunes'),
 (0.006581992983594227, '2018'),
 (0.006397769664972892, 'performing'),
 (0.00621785726109142, 'social'),
 (0.006007109268014441, 'nitunes'),
 (0.0059850753586936966, 'craft'),
 (0.005394057696382439, 'pons'),
 (0.005060474061676277, 'glue'),
 (0.004949674970851037, 'diy'),
 (0.004797908756927949, 'episode'),
 (0.00475896988795098, 'stay')]

In [0]:
## Predicting whole
y_pred = rf_model.predict(x_test)
precision,recall,fscore,support =score(y_test, y_pred, pos_label='category_id', average=None)

/databricks/python/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'category_id') is ignored when average != 'binary' (got None). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/databricks/python/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
## Accuracy Whole
import numpy as np
print ('Accuracy {}'  .format (np. round ((y_pred == y_test).sum ()/len (y_pred),2)))

Accuracy 0.65


###RF predictor on BoW on new_pred

In [0]:
new_counts = ngram_vect.transform([new_pred])
prediction_rf = rf_model.predict(new_counts)
prediction_rf

Out[84]: array(['10'], dtype=object)

###Naive Bayes on Bag of Words Features

In [0]:
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [0]:
Naive = naive_bayes.MultinomialNB()
Naive.fit(x_train, y_train)
# predict the labels on validation dataset
predictions_NB = Naive.predict(x_test)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, y_test)*100)

Naive Bayes Accuracy Score ->  78.3132530120482


### NB predictors

In [0]:
new_counts = ngram_vect.transform([new_pred])
prediction = Naive.predict(new_counts)
prediction

Out[85]: array(['24'], dtype='<U2')

### SVM on Bag of Words Features

In [0]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(x_train, y_train)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(x_test)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, y_test)*100)

SVM Accuracy Score ->  75.90361445783132


In [0]:
#ngram_vect = CountVectorizer (ngram_range=(1,1))
#x_count_ngram_lemm = pd.DataFrame (ngram_vect.fit_transform (df['text_lemmatized_ngram_join']).toarray())
#print (x_count_ngram_lemm.shape)
#print (ngram_vect.get_feature_names())
#x_count_ngram_lemm.columns =ngram_vect.get_feature_names()
#x_count_ngram_lemm.head ()

new_counts = ngram_vect.fit_transform([new_pred])
prediction_svm = SVM.predict(new_counts)
prediction_svm

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<command-2697584825671114> in <cell line: 9>()
      7 
      8 new_counts = ngram_vect.fit_transform([new_pred])
----> 9 prediction_svm = SVM.predict(new_counts)
     10 prediction_svm

/databricks/python/lib/python3.9/site-packages/sklearn/svm/_base.py in predict(self, X)
    622             y = np.argmax(self.decision_function(X), axis=1)
    623         else:
--> 624             y = super().predict(X)
    625         return self.classes_.take(np.asarray(y, dtype=np.intp))
    626 

/databricks/python/lib/python3.9/site-packages/sklearn/svm/_base.py in predict(self, X)
    340         y_pred : ndarray of shape (n_samples,)
    341         """
--> 342         X = self._validate_for_predict(X)
    343         predict = self._sparse_predict if self._sparse else self._dense_predict
    344         return predict(X)

/databricks/python/li

###Gradient Boosting Classifier on Bag of Words Features

In [0]:
from sklearn.ensemble import GradientBoostingClassifier

gb_clf = GradientBoostingClassifier(n_estimators=20, learning_rate=0.25, max_features=2, max_depth=2, random_state=0)
gb_clf.fit(x_train, y_train)

predictions_GB = gb_clf.predict(x_test)
print("Gradient Boosting Classifier -> ",accuracy_score(predictions_GB, y_test)*100)

Gradient Boosting Classifier ->  46.08433734939759


In [0]:
lr_list = [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]

for learning_rate in lr_list:
    gb_clf = GradientBoostingClassifier(n_estimators=20, learning_rate=learning_rate, max_features=2, max_depth=2, random_state=0)
    gb_clf.fit(x_train, y_train)
    predictions_GB = gb_clf.predict(x_test)
    
    print("Learning rate: ", learning_rate)
    print("Gradient Boosting Classifier -> ", accuracy_score(predictions_GB, y_test)*100)

Learning rate:  0.05
Gradient Boosting Classifier ->  44.87951807228915
Learning rate:  0.075
Gradient Boosting Classifier ->  44.87951807228915
Learning rate:  0.1
Gradient Boosting Classifier ->  44.87951807228915
Learning rate:  0.25
Gradient Boosting Classifier ->  46.08433734939759
Learning rate:  0.5
Gradient Boosting Classifier ->  45.48192771084337
Learning rate:  0.75
Gradient Boosting Classifier ->  44.27710843373494
Learning rate:  1
Gradient Boosting Classifier ->  44.27710843373494


###KNN on Bag of Words Features

In [0]:
from sklearn.neighbors import KNeighborsClassifier

In [0]:
KNN_ = KNeighborsClassifier(n_neighbors=100)
KNN_.fit(x_train,y_train)

predictions_KNN = gb_clf.predict(x_test)
print("K Nearest Classifier -> ",accuracy_score(predictions_KNN, y_test)*100)

K Nearest Classifier ->  46.08433734939759


In [0]:
new_counts = ngram_vect.fit_transform([new_pred])
prediction_knn = KNN_.predict(new_counts)
prediction_knn

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<command-2697584825671118> in <cell line: 2>()
      1 new_counts = ngram_vect.fit_transform([new_pred])
----> 2 prediction_knn = KNN_.predict(new_counts)
      3 prediction_knn

/databricks/python/lib/python3.9/site-packages/sklearn/neighbors/_classification.py in predict(self, X)
    195         X = check_array(X, accept_sparse='csr')
    196 
--> 197         neigh_dist, neigh_ind = self.kneighbors(X)
    198         classes_ = self.classes_
    199         _y = self._y

/databricks/python/lib/python3.9/site-packages/sklearn/neighbors/_base.py in kneighbors(self, X, n_neighbors, return_distance)
    703                 kwds = self.effective_metric_params_
    704 
--> 705             chunked_results = list(pairwise_distances_chunked(
    706                 X, self._fit_X, reduce_func=reduce_func,
    707                 metric=self.e

###TF-IDF Features Extraction

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
Tfidf_vect = TfidfVectorizer(max_features=23482)
Tfidf_vect.fit(df['text_lemmatized_ngram_join'])

x_tfidf_ngram_lemm = pd.DataFrame (Tfidf_vect.fit_transform (df ['text_lemmatized_ngram_join']).toarray())
print (x_tfidf_ngram_lemm.shape)
print (Tfidf_vect.get_feature_names())
x_tfidf_ngram_lemm.columns = Tfidf_vect.get_feature_names()
x_tfidf_ngram_lemm.head (100)

(1660, 22464)
['00', '000', '000fps', '000mah', '00am', '00pm', '01', '011', '02', '0211have', '02new', '03', '04', '0410', '05', '05062018series', '06', '07', '07062018series', '08', '08pm', '09', '09717072065', '09811946306', '09821297834', '099', '09traveling', '10', '100', '1000', '10005503', '10008740', '1000fps', '10019released', '10037341', '10037342', '10037343', '10037344', '10037345', '10037346', '10041959', '10053660', '10053666', '10053673', '10053683', '10074086', '10080709', '10080725', '10080727', '10080742', '10093711', '100k', '101', '10102320', '10110895', '10140258', '10141660', '10146922', '10146924', '10151856', '10151874', '10151881', '102', '10212204', '10212205', '10229357', '10229358', '10229359', '10229360', '10241097', '10241098', '10241099', '10241117', '10268440', '10268441', '10268444', '10268445', '10268547', '10293642', '10293679', '10293684', '102notoutteaser', '103', '10312882', '10312884', '10312885', '10332196', '10332198', '10332199', '10332200', '1

,00,000,000fps,000mah,00am,00pm,01,011,02,0211have,...,zsuzsanna,zubaan,zuckerberg,zuniga,zur,zurita,zwadlo,zx110,zyaada,zyll
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
x_train_tfidf, x_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(x_tfidf_ngram_lemm, df['category_id'], test_size=0.2)

###RF on Tf_idf features

In [0]:
# fitting the model whole
rf_tf = RandomForestClassifier(n_estimators=50, max_depth=20, n_jobs=-1)
rf_model_tf = rf_tf.fit(x_train_tfidf, y_train_tfidf)

In [0]:
## Predicting
y_pred_tfidf = rf_model.predict(x_test_tfidf)
precision,recall,fscore,support = score(y_test_tfidf, y_pred_tfidf, pos_label='category_id', average=None)

/databricks/python/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'category_id') is ignored when average != 'binary' (got None). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/databricks/python/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
## Accuracy Whole
import numpy as np
print ('Accuracy {}'  .format (np. round ((y_pred_tfidf == y_test_tfidf).sum ()/len (y_pred_tfidf),2)))

Accuracy 0.44


In [0]:
new_counts_tf = Tfidf_vect.transform([new_pred])
prediction_rf_tf = rf_model.predict(new_counts_tf)
prediction_rf_tf

Out[90]: array(['10'], dtype=object)

### NB on TFIDF

In [0]:
Naive_tfidf = naive_bayes.MultinomialNB()
Naive_tfidf.fit(x_train_tfidf, y_train_tfidf)
# predict the labels on validation dataset
predictions_NB_tfidf = Naive_tfidf.predict(x_test_tfidf)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score with tfidf features -> ",accuracy_score(predictions_NB_tfidf, y_test_tfidf)*100)

Naive Bayes Accuracy Score with tfidf features ->  66.86746987951807


In [0]:
new_counts_tf = Tfidf_vect.transform([new_pred])
prediction_nb_tf = Naive_tfidf.predict(new_counts_tf)
prediction_nb_tf

Out[92]: array(['10'], dtype='<U2')

###SVM on TF-IDF Features

In [0]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(x_train_tfidf, y_train_tfidf)
# predict the labels on validation dataset
predictions_SVM_tfidf = SVM.predict(x_test_tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score with tfidf features -> ",accuracy_score(predictions_SVM_tfidf, y_test_tfidf)*100)

SVM Accuracy Score with tfidf features ->  78.6144578313253


###Random Forest Classifier in Tf_idf features

In [0]:
rf_1=RandomForestClassifier(n_estimators=50, max_depth=75, n_jobs=-1)
rf_model = rf_1.fit(x_train_tfidf, y_train_tfidf)

predictions_rf_tfidf = rf_model.predict(x_test_tfidf)

print("Random Forest Classifier Accuracy Score with tfidf features -> ",accuracy_score(predictions_rf_tfidf, y_test_tfidf)*100)

Random Forest Classifier Accuracy Score with tfidf features ->  76.20481927710844


In [0]:
new_counts_tf = Tfidf_vect.transform([new_pred])
prediction_rf_tf = rf_model.predict(new_counts_tf)
prediction_rf_tf

Out[73]: array(['24'], dtype=object)